In [1]:
import cv2
import numpy as np
import serial
import time
from IPython.display import Audio

#Audio
cheering = 'cheering.mp3'

# Color ranges and grid size (same as before)
color_ranges = {
    'white': ([0, 0, 200], [180, 30, 255]),
    'yellow': ([20, 100, 100], [40, 255, 255]),
    'blue': ([100, 100, 100], [130, 255, 255]),
    'green': ([50, 100, 100], [80, 255, 255]),
    'orange': ([10, 100, 100], [25, 255, 255]), # Old values: ([5, 100, 100], [15, 255, 255])
    'lower_red': ([0, 100, 100], [7, 255, 255]),   # Old values: ([0, 100, 100], [10, 255, 255])
    'upper_red': ([170, 100, 100], [180, 255, 255])
}
GRID_SIZE = 3

# Serial port configuration
port = 'COM4'  # Replace with your COM port
baudrate = 115200

try:
    ser = serial.Serial(port, baudrate, timeout=1)
    print(f"Connected to {port} at {baudrate} baud")
except serial.SerialException as e:
    print(f"Error: Could not open serial port {port}: {e}")
    ser = None


def display_image(image_path, window_name="Image"):
    """
    Opens a window and displays the image from the given path.

    Args:
        image_path (str): The path to the image file.
        window_name (str, optional): The name of the window. Defaults to "Image".
    """
    try:
        img = cv2.imread(image_path)
        if img is not None:
            cv2.imshow(window_name, img)
            # cv2.waitKey(0)  # Wait until a key is pressed
            # cv2.destroyWindow(window_name) # Close the window
        else:
            print(f"Error: Could not open or find the image at {image_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

def detect_colors(frame, roi):
    # (Same as before)
    x, y, w, h = roi
    face_roi = frame[y:y + h, x:x + w]
    if face_roi.size == 0:  # Check if face_roi is empty
        return None

    # Add the Gaussian blur here
    face_roi = cv2.GaussianBlur(face_roi, (5, 5), 0)
    
    face_hsv = cv2.cvtColor(face_roi, cv2.COLOR_BGR2HSV)
    cell_width = w // GRID_SIZE
    cell_height = h // GRID_SIZE
    detected_colors = []

    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            cell_x = j * cell_width
            cell_y = i * cell_height
            cell = face_hsv[cell_y:cell_y + cell_height, cell_x:cell_x + cell_width]
            if cell.size > 0:
                avg_hsv = np.mean(cell.reshape(-1, 3), axis=0)
                hue, sat, val = avg_hsv

                detected_color = None
                for color_name, (lower, upper) in color_ranges.items():
                    lower_np = np.array(lower)
                    upper_np = np.array(upper)
                    if color_name in ['lower_red', 'upper_red']:
                        if (hue >= lower_np[0] and hue <= upper_np[0] and
                                sat >= lower_np[1] and sat <= upper_np[1] and
                                val >= lower_np[2] and val <= upper_np[2]):
                            detected_color = 'red'
                            break
                    elif (hue >= lower_np[0] and hue <= upper_np[0] and
                          sat >= lower_np[1] and sat <= upper_np[1] and
                          val >= lower_np[2] and val <= upper_np[2]):
                        detected_color = color_name
                        break
                detected_colors.append(detected_color)
            else:
                return None
    return detected_colors

def is_face_solved(colors):
    # (Same as before)
    if not colors or None in colors:
        return False
    first_color = colors[0]
    return all(color == first_color for color in colors)

def main():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open camera.")
        return

    roi_x, roi_y, roi_w, roi_h = 100, 100, 300, 300
    drawing = False
    start_point = None
    solved_faces = 0
    last_solved_time = 0  # Initialize the last solved detection time
    delay_seconds = 1.5     # Set the delay in seconds
    no_solve_timeout = 7
    last_check_time = time.time()
    detecting = False  # Flag to control detection

    def draw_roi(event, x, y, flags, param):
        # (Same as before)
        nonlocal roi_x, roi_y, roi_w, roi_h, drawing, start_point
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            start_point = (x, y)
        elif event == cv2.EVENT_MOUSEMOVE:
            if drawing:
                pass
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            if start_point:
                roi_x = min(start_point[0], x)
                roi_y = min(start_point[1], y)
                roi_w = abs(start_point[0] - x)
                roi_h = abs(start_point[1] - y)
                start_point = None

    cv2.namedWindow("Rubik's Cube Solver")
    cv2.setMouseCallback("Rubik's Cube Solver", draw_roi)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_w, roi_y + roi_h), (0, 255, 0), 2)


        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            if ser:
                ser.write(b"closing\r\n")
            break
        elif key == ord('r'):
            roi_x, roi_y, roi_w, roi_h = 100, 100, 200, 200
        elif key == ord('s'):
            if ser:
                ser.write(b"solved\r\n")
            solved_faces += 1
            print(f"Solved Faces: {solved_faces}")
            last_solved_time = current_frame_time  # Update the last solved time
            last_check_time = current_frame_time # Reset the timer if a face is solved
        elif key == ord(' '):
            detecting = not detecting  # Toggle detection on spacebar press
            if detecting:
                print("Detection started/resumed.")
            else:
                print("Detection paused.")
            last_check_time = time.time() # Reset the timeout timer when starting/resuming

        if detecting:

            if detecting and (key == ord('p')): # Press 'p' to print center pixel HSV
                center_x = roi_x + roi_w // 2
                center_y = roi_y + roi_h // 2
                hsv_center = cv2.cvtColor(frame[center_y, center_x][np.newaxis, np.newaxis, :], cv2.COLOR_BGR2HSV)[0][0]
                print(f"Center pixel HSV: {hsv_center}")
        
            # Check if 4 sides are solved
            if(solved_faces >= 4):
                print("You solved the rubiks cube!!!")
                if ser:
                    ser.write(b"fullysolved\r\n")
                display_image('celebration.webp', "Hooray!")
                solved_faces = 0
                detecting = False
    
            detected_face_colors = detect_colors(frame, (roi_x, roi_y, roi_w, roi_h))
            current_frame_time = time.time()
    
            if detected_face_colors:
                # (Same as before)
                cell_width = roi_w // GRID_SIZE
                cell_height = roi_h // GRID_SIZE
                for i in range(GRID_SIZE):
                    for j in range(GRID_SIZE):
                        cell_x = roi_x + j * cell_width
                        cell_y = roi_y + i * cell_height
                        color_name = detected_face_colors[i * GRID_SIZE + j]
                        if color_name:
                            color_bgr = None
                            if color_name == 'white':
                                color_bgr = (255, 255, 255)
                            elif color_name == 'yellow':
                                color_bgr = (0, 255, 255)
                            elif color_name == 'blue':
                                color_bgr = (255, 0, 0)
                            elif color_name == 'orange':
                                color_bgr = (0, 165, 255)
                            elif color_name == 'green':
                                color_bgr = (0, 255, 0)
                            elif color_name == 'red':
                                color_bgr = (0, 0, 255)
    
                            if color_bgr:
                                cv2.rectangle(frame, (cell_x, cell_y),
                                              (cell_x + cell_width, cell_y + cell_height),
                                              color_bgr, -1)
                                cv2.putText(frame, color_name, (cell_x + 5, cell_y + cell_height - 5),
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
                        else:
                            cv2.putText(frame, "No Color", (cell_x + 5, cell_y + cell_height - 5),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
    
                if is_face_solved(detected_face_colors):
                    if current_frame_time - last_solved_time >= delay_seconds:
                        cv2.putText(frame, "Face Solved!", (roi_x, roi_y - 20),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                        
                        
                        if ser:
                            ser.write(b"solved\r\n")
                            solved_faces += 1
                            print(f"Solved Faces: {solved_faces}")
                            Audio(cheering)
                            last_solved_time = current_frame_time  # Update the last solved time
                    # Removed the else here for the waiting message to avoid confusion with the timeout
                else:
                    cv2.putText(frame, "Face Not Solved", (roi_x, roi_y - 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
            else:
                cv2.putText(frame, "No Face Detected", (roi_x, roi_y - 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
            # Check for no solve timeout EVERY FRAME
            if current_frame_time - last_check_time >= no_solve_timeout:
                if ser:
                    ser.write(b"failed\r\n")
                print(f"No solved face detected for {no_solve_timeout} seconds.")
                print("YOU FAILED!!!!")
                display_image('failed.webp', "SOLVE THE CUBE!")
                solved_faces = 0
                detecting = False
                last_check_time = current_frame_time # Reset the timer
            elif detected_face_colors and is_face_solved(detected_face_colors):
                last_check_time = current_frame_time # Reset the timer if a face is solved

        cv2.imshow("Rubik's Cube Solver", frame)

        

    cap.release()
    cv2.destroyAllWindows()
    if ser and ser.is_open:
        ser.close()
        print("Serial port closed.")

if __name__ == "__main__":
    main()

Error: Could not open serial port COM4: could not open port 'COM4': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)
Detection started/resumed.
